In [ ]:
import requests
import os
import base64
import csv
import time
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Spotify API credentials
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

# Discogs API credentials
discogs_token = os.getenv('discogs_token')
user_agent = os.getenv('USER_AGENT')

# Validate credentials
if not CLIENT_ID or not CLIENT_SECRET or not discogs_token or not user_agent:
    raise ValueError("API credentials or User-Agent are missing! Please check your .env file.")

def top_album_studios(limit, min_yr, max_yr, mrkt):
    """
    Fetch top albums from Spotify and retrieve their recording studios from Discogs.
    Results are saved to a CSV file.
    
    Parameters:
        limit (int): Total number of albums to fetch per year from Spotify (up to 1,000).
        min_yr (int): Minimum year to fetch albums for.
        max_yr (int): Maximum year to fetch albums for.
        mrkt (str): Spotify market (e.g., "US").
    """
    def get_access_token():
        """Obtain an access token using Spotify Client Credentials Flow."""
        url = "https://accounts.spotify.com/api/token"
        headers = {
            "Authorization": "Basic " + base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()
        }
        data = {"grant_type": "client_credentials"}
        response = requests.post(url, headers=headers, data=data)
        return response.json().get("access_token")

    def fetch_albums_by_years(limit, min_yr, max_yr, mrkt):
        """Fetch albums by year range from Spotify with dynamic pagination."""
        access_token = get_access_token()
        all_albums = []

        # Enforce Spotify's query limit of 1,000 albums per query
        if limit > 1000:
            raise ValueError("Spotify API limits queries to 1,000 results per year.")

        for year in range(min_yr, max_yr + 1):
            print(f"Fetching up to {limit} albums for the year {year}...")
            offset = 0
            albums_fetched = 0

            while albums_fetched < limit:
                batch_limit = min(limit - albums_fetched, 50)  # Spotify allows up to 50 results per request
                url = "https://api.spotify.com/v1/search"
                headers = {"Authorization": f"Bearer {access_token}"}
                params = {
                    "q": f"year:{year}",
                    "type": "album",
                    "limit": batch_limit,
                    "offset": offset,
                    "market": mrkt
                }
                response = requests.get(url, headers=headers, params=params)
                if response.status_code != 200:
                    print(f"Failed to fetch albums for {year}: {response.status_code} - {response.text}")
                    break

                albums = response.json().get('albums', {}).get('items', [])
                if not albums:
                    break  # No more albums to fetch

                for album in albums:
                    album_name = album.get('name', 'Unknown Album')
                    artist_name = album.get('artists', [{}])[0].get('name', 'Unknown Artist')
                    all_albums.append((album_name, artist_name, year))
                    albums_fetched += 1

                offset += batch_limit  # Update offset for the next batch

                # Exit loop early if we've fetched the requested number of albums
                if albums_fetched >= limit:
                    break

        return all_albums

    def get_all_versions(base_url, master_id, headers):
        """Get all versions of a master release from Discogs."""
        if not master_id:
            return []
        versions_url = f"{base_url}/masters/{master_id}/versions"
        versions_response = requests.get(versions_url, headers=headers)
        return versions_response.json().get('versions', [])

    def get_recording_studios(album_list, discogs_token):
        """Fetch recording studios for albums using the Discogs API."""
        headers = {
            'Authorization': f'Discogs token={discogs_token}',
            'User-Agent': user_agent
        }
        base_url = 'https://api.discogs.com'
        releases = []

        print("Starting to process albums...")

        for index, (album_name, artist_name, year) in enumerate(album_list, 1):
            print(f"\nChecking [{index}]: {artist_name} - {album_name} ({year})")
            search_url = f"{base_url}/database/search"
            params = {'q': album_name, 'artist': artist_name, 'type': 'release'}
            search_response = requests.get(search_url, headers=headers, params=params)
            search_results = search_response.json()
            if not search_results.get('results'):
                print(f"No results found on Discogs for: {album_name} by {artist_name}")
                continue

            release_id = search_results['results'][0].get('id')
            release_url = f"{base_url}/releases/{release_id}"
            time.sleep(1)  # Respect rate limits
            release_response = requests.get(release_url, headers=headers)
            release_data = release_response.json()

            recording_studios = set()
            master_id = release_data.get('master_id')

            if master_id:
                master_url = f"{base_url}/masters/{master_id}"
                master_response = requests.get(master_url, headers=headers)
                master_data = master_response.json()
                for company in master_data.get('companies', []):
                    if company.get('entity_type_name') == 'Recorded At':
                        recording_studios.add(company.get('name', ''))

                versions = get_all_versions(base_url, master_id, headers)
                for version in versions:
                    version_id = version.get('id')
                    version_url = f"{base_url}/releases/{version_id}"
                    time.sleep(1)
                    version_response = requests.get(version_url, headers=headers)
                    version_data = version_response.json()
                    for company in version_data.get('companies', []):
                        if company.get('entity_type_name') == 'Recorded At':
                            recording_studios.add(company.get('name', ''))

            for company in release_data.get('companies', []):
                if company.get('entity_type_name') == 'Recorded At':
                    recording_studios.add(company.get('name', ''))

            releases.append({
                'artist': artist_name,
                'album_name': album_name,
                'recorded_at': '; '.join(sorted(recording_studios)) if recording_studios else 'Not specified',
                'year': year
            })

        output_file = f'top_album_studios_{min_yr}_{max_yr}.csv'
        with open(output_file, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['artist', 'album_name', 'recorded_at', 'year'])
            writer.writeheader()
            writer.writerows(releases)

        print(f"\nData successfully exported to {output_file}")
        return output_file

    # Fetch albums from Spotify
    albums = fetch_albums_by_years(limit, min_yr, max_yr, mrkt)

    # Fetch recording studios from Discogs and export to CSV
    return get_recording_studios(albums, discogs_token)

In [ ]:
top_album_studios(300, 2019, 2019, "US")

In [ ]:
import requests
import os
import base64
import csv
import time
from datetime import datetime
from dotenv import load_dotenv
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Load environment variables from .env file
load_dotenv()

# Spotify API credentials
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

# Discogs API credentials
discogs_token = os.getenv('discogs_token')
user_agent = os.getenv('USER_AGENT')

# Validate credentials
if not CLIENT_ID or not CLIENT_SECRET or not discogs_token or not user_agent:
    raise ValueError("API credentials or User-Agent are missing! Please check your .env file.")

# Configure retry strategy
def create_session():
    session = requests.Session()
    retries = Retry(
        total=5,  # number of retries
        backoff_factor=1,  # wait 1, 2, 4, 8, 16 seconds between retries
        status_forcelist=[429, 500, 502, 503, 504]  # HTTP status codes to retry on
    )
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

def make_discogs_request(url, headers, params=None):
    """Make a request to Discogs API with rate limiting handling"""
    session = create_session()
    try:
        response = session.get(url, headers=headers, params=params)
        
        # Check rate limiting headers
        remaining = int(response.headers.get('X-Discogs-Ratelimit-Remaining', 0))
        
        # If we're running low on requests, wait longer
        if remaining < 10:
            time.sleep(3)  # Wait 3 seconds when close to limit
        else:
            time.sleep(1)  # Standard 1 second wait
            
        if response.status_code == 429:
            retry_after = int(response.headers.get('Retry-After', 60))
            print(f"Rate limited. Waiting {retry_after} seconds...")
            time.sleep(retry_after)
            return make_discogs_request(url, headers, params)
            
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error making request to {url}: {str(e)}")
        return None

def top_album_studios(limit, min_yr, max_yr, mrkt):
    """
    Fetch top albums from Spotify and retrieve their recording studios from Discogs.
    Results are saved to a CSV file.
    """
    def get_access_token():
        """Obtain an access token using Spotify Client Credentials Flow."""
        session = create_session()
        url = "https://accounts.spotify.com/api/token"
        headers = {
            "Authorization": "Basic " + base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()
        }
        data = {"grant_type": "client_credentials"}
        response = session.post(url, headers=headers, data=data)
        return response.json().get("access_token")

    def fetch_albums_by_years(limit, min_yr, max_yr, mrkt):
        """Fetch albums by year range from Spotify with dynamic pagination."""
        session = create_session()
        access_token = get_access_token()
        all_albums = []

        if limit > 1000:
            raise ValueError("Spotify API limits queries to 1,000 results per year.")

        for year in range(min_yr, max_yr + 1):
            print(f"Fetching up to {limit} albums for {year}...")
            offset = 0
            albums_fetched = 0

            while albums_fetched < limit:
                batch_limit = min(limit - albums_fetched, 50)
                url = "https://api.spotify.com/v1/search"
                headers = {"Authorization": f"Bearer {access_token}"}
                params = {
                    "q": f"year:{year}",
                    "type": "album",
                    "limit": batch_limit,
                    "offset": offset,
                    "market": mrkt
                }
                
                try:
                    response = session.get(url, headers=headers, params=params)
                    response.raise_for_status()
                    data = response.json()
                    
                    albums = data.get('albums', {}).get('items', [])
                    if not albums:
                        break

                    for album in albums:
                        album_name = album.get('name', 'Unknown Album')
                        artist_name = album.get('artists', [{}])[0].get('name', 'Unknown Artist')
                        all_albums.append((album_name, artist_name, year))
                        albums_fetched += 1

                    offset += batch_limit
                    
                    if albums_fetched >= limit:
                        break
                        
                    time.sleep(1)  # Rate limiting for Spotify
                    
                except requests.exceptions.RequestException as e:
                    print(f"Error fetching albums for {year}: {str(e)}")
                    break

        return all_albums

    def get_all_versions(base_url, master_id, headers):
        """Get all versions of a master release from Discogs."""
        if not master_id:
            return []
        versions_url = f"{base_url}/masters/{master_id}/versions"
        return make_discogs_request(versions_url, headers, {}).get('versions', [])

    def get_recording_studios(album_list, discogs_token):
        """Fetch recording studios for albums using the Discogs API."""
        headers = {
            'Authorization': f'Discogs token={discogs_token}',
            'User-Agent': user_agent
        }
        base_url = 'https://api.discogs.com'
        releases = []

        print("Starting to process albums...")

        for index, (album_name, artist_name, year) in enumerate(album_list, 1):
            print(f"\nChecking [{index}]: {artist_name} - {album_name} ({year})")
            
            # Search for release
            search_url = f"{base_url}/database/search"
            params = {'q': album_name, 'artist': artist_name, 'type': 'release'}
            search_results = make_discogs_request(search_url, headers, params)
            
            if not search_results or not search_results.get('results'):
                print(f"No results found on Discogs for: {album_name} by {artist_name}")
                continue

            release_id = search_results['results'][0].get('id')
            release_url = f"{base_url}/releases/{release_id}"
            release_data = make_discogs_request(release_url, headers)
            
            if not release_data:
                continue

            recording_studios = set()
            master_id = release_data.get('master_id')

            if master_id:
                master_url = f"{base_url}/masters/{master_id}"
                master_data = make_discogs_request(master_url, headers)
                
                if master_data:
                    for company in master_data.get('companies', []):
                        if company.get('entity_type_name') == 'Recorded At':
                            recording_studios.add(company.get('name', ''))

                    versions = get_all_versions(base_url, master_id, headers)
                    for version in versions:
                        version_id = version.get('id')
                        version_url = f"{base_url}/releases/{version_id}"
                        version_data = make_discogs_request(version_url, headers)
                        
                        if version_data:
                            for company in version_data.get('companies', []):
                                if company.get('entity_type_name') == 'Recorded At':
                                    recording_studios.add(company.get('name', ''))

            for company in release_data.get('companies', []):
                if company.get('entity_type_name') == 'Recorded At':
                    recording_studios.add(company.get('name', ''))

            releases.append({
                'artist': artist_name,
                'album_name': album_name,
                'recorded_at': '; '.join(sorted(recording_studios)) if recording_studios else 'Not specified',
                'year': year
            })

        output_file = f'top_album_studios_{min_yr}_{max_yr}.csv'
        with open(output_file, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['artist', 'album_name', 'recorded_at', 'year'])
            writer.writeheader()
            writer.writerows(releases)

        print(f"\nData successfully exported to {output_file}")
        return output_file

    albums = fetch_albums_by_years(limit, min_yr, max_yr, mrkt)
    return get_recording_studios(albums, discogs_token)

In [ ]:
top_album_studios(300, 2020, 2021, "US")

In [2]:
import requests
import os
import base64
import csv
import time
from datetime import datetime
import concurrent.futures
from functools import lru_cache
from typing import Dict, List, Set, Optional
from dotenv import load_dotenv
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Load environment variables from .env file
load_dotenv()

# Spotify API credentials
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
discogs_token = os.getenv('discogs_token')
user_agent = os.getenv('USER_AGENT')

# Validate credentials
if not CLIENT_ID or not CLIENT_SECRET or not discogs_token or not user_agent:
    raise ValueError("API credentials or User-Agent are missing! Please check your .env file.")

# Configure session with connection pooling
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504]
)
adapter = HTTPAdapter(
    max_retries=retries,
    pool_connections=20,  # Increase connection pool size
    pool_maxsize=20
)
session.mount('https://', adapter)

# Cache for API responses
@lru_cache(maxsize=1000)
def cached_discogs_request(url: str, token: str) -> Optional[Dict]:
    """Cache Discogs API responses to avoid duplicate requests"""
    headers = {
        'Authorization': f'Discogs token={token}',
        'User-Agent': user_agent
    }
    try:
        response = session.get(url, headers=headers)
        remaining = int(response.headers.get('X-Discogs-Ratelimit-Remaining', 0))
        
        if remaining < 5:
            time.sleep(2)
        
        if response.status_code == 429:
            time.sleep(int(response.headers.get('Retry-After', 60)))
            return cached_discogs_request(url, token)
            
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException:
        return None

def get_spotify_access_token() -> str:
    """Get Spotify access token with caching"""
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()
    }
    data = {"grant_type": "client_credentials"}
    response = session.post(url, headers=headers, data=data)
    return response.json().get("access_token")

def fetch_spotify_albums(year: int, limit: int, offset: int, access_token: str, market: str) -> List:
    """Fetch a batch of albums from Spotify"""
    url = "https://api.spotify.com/v1/search"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {
        "q": f"year:{year}",
        "type": "album",
        "limit": min(limit, 50),
        "offset": offset,
        "market": market
    }
    
    response = session.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json().get('albums', {}).get('items', [])

def process_album(album_data: tuple) -> Dict:
    """Process a single album to find recording studios"""
    album_name, artist_name, year = album_data
    base_url = 'https://api.discogs.com'
    recording_studios = set()
    
    # Search for release
    search_url = f"{base_url}/database/search"
    params = {'q': f"{album_name} {artist_name}", 'type': 'release'}
    search_results = cached_discogs_request(f"{search_url}?{requests.compat.urlencode(params)}", discogs_token)
    
    if not search_results or not search_results.get('results'):
        return {
            'artist': artist_name,
            'album_name': album_name,
            'recorded_at': 'Not found',
            'year': year
        }

    # Process first 3 most relevant results instead of just the first one
    for result in search_results['results'][:3]:
        release_id = result.get('id')
        release_data = cached_discogs_request(f"{base_url}/releases/{release_id}", discogs_token)
        
        if not release_data:
            continue

        # Extract studios from release
        for company in release_data.get('companies', []):
            if company.get('entity_type_name') == 'Recorded At':
                recording_studios.add(company.get('name', ''))

        # Check master release if exists
        master_id = release_data.get('master_id')
        if master_id:
            master_data = cached_discogs_request(f"{base_url}/masters/{master_id}", discogs_token)
            if master_data:
                for company in master_data.get('companies', []):
                    if company.get('entity_type_name') == 'Recorded At':
                        recording_studios.add(company.get('name', ''))

    return {
        'artist': artist_name,
        'album_name': album_name,
        'recorded_at': '; '.join(sorted(recording_studios)) if recording_studios else 'Not specified',
        'year': year
    }

def top_album_studios(limit: int, min_yr: int, max_yr: int, mrkt: str) -> str:
    """Main function to fetch albums and their recording studios"""
    all_albums = []
    access_token = get_spotify_access_token()
    
    # Fetch albums from Spotify
    for year in range(min_yr, max_yr + 1):
        print(f"Fetching albums for {year}...")
        offset = 0
        while len(all_albums) < limit:
            batch = fetch_spotify_albums(year, min(50, limit - len(all_albums)), offset, access_token, mrkt)
            if not batch:
                break
                
            all_albums.extend([
                (album.get('name', 'Unknown Album'),
                 album.get('artists', [{}])[0].get('name', 'Unknown Artist'),
                 year)
                for album in batch
            ])
            offset += len(batch)
            time.sleep(0.5)  # Reduced sleep time for Spotify API
    
    # Process albums in parallel
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_to_album = {executor.submit(process_album, album): album for album in all_albums}
        for future in concurrent.futures.as_completed(future_to_album):
            album = future_to_album[future]
            try:
                result = future.result()
                results.append(result)
                print(f"Processed: {result['artist']} - {result['album_name']}")
            except Exception as e:
                print(f"Error processing {album}: {str(e)}")
    
    # Write results to CSV
    output_file = f'top_album_studios_{min_yr}_{max_yr}.csv'
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['artist', 'album_name', 'recorded_at', 'year'])
        writer.writeheader()
        writer.writerows(results)
    
    return output_file

In [3]:
# Example usage
result_file = top_album_studios(limit=900, min_yr=2022,max_yr=2024,mrkt='US')

Fetching albums for 2022...
Fetching albums for 2023...
Fetching albums for 2024...
Processed: Metro Boomin - HEROES & VILLAINS
Processed: Zach Bryan - American Heartbreak
Processed: SZA - SOS
Processed: Bad Bunny - Un Verano Sin Ti
Processed: Noah Kahan - Stick Season
Processed: Harry Styles - Harry's House
Processed: SZA - SOS
Processed: Taylor Swift - Midnights
Processed: Sabrina Carpenter - emails i can't send
Processed: Kendrick Lamar - Mr. Morale & The Big Steppers
Processed: Drake - Her Loss
Processed: Juice WRLD - Fighting Demons (Deluxe)
Processed: Milli Hughes - Deep Phase Noise
Processed: beabadoobee - Beatopia
Processed: Future - I NEVER LIKED YOU
Processed: Brent Faiyaz - WASTELAND
Processed: Zach Bryan - Something in the Orange
Processed: Zach Bryan - Summertime Blues
Processed: Steve Lacy - Gemini Rights
Processed: $uicideboy$ - Sing Me a Lullaby, My Sweet Temptation
Processed: Laufey - Everything I Know About Love
Processed: Djo - DECIDE
Processed: Various Artists - 100

Processed: Kelsea Ballerini - SUBJECT TO CHANGE
Processed: Mora - MICRODOSIS
Processed: Destroy Lonely - NS+ (ULTRA)
Processed: SleazyWorld Go - Where The Shooters Be
Processed: Mitski - Laurel Hell
Processed: Stray Kids - ODDINARY
Processed: Cris Mj - Una Noche en Medellín
Processed: The Backseat Lovers - Waiting to Spill
Processed: Luis R Conriquez - El Gavilán
Processed: Yeat - 2 Alivë (Geëk Pack)
Processed: Kanye West - True Love
Processed: That Mexican OT - Nonsense and Mexican Shit
Processed: Take Care - Reject
Processed: Joey Bada$$ - 2000
Processed: Junior H - CONTINGENTE
Processed: Zach Bryan - The Greatest Day of My Life
Processed: Tory Lanez - Sorry 4 What
Processed: Jane Color - Relaxing Brown Noise 256 Hz
Processed: Alec Benjamin - (Un)Commentary
Processed: Zach Bryan - All My Homies Hate Ticketmaster (Live from Red Rocks)
Processed: Yatta Bandz - Perfect Storm
Processed: Evomin - White Noise Comfort
Processed: Yuridia - Pa' Luego Es Tarde
Processed: Midrift - Midrift
Proc

Processed: Øneheart - this feeling
Processed: Kim Petras - Slut Pop
Processed: Depeche Mode - Violator | The 12" Singles
Processed: Morgan Wallen - You Proof
Processed: Fall Asleep Machine - Full Focus
Processed: daniel.mp3 - green to blue (slowed + reverbed)
Processed: Soft Soundscapes - Sanfter Regen
Processed: Kordhell - Murder In My Mind
Processed: Lorna Shore - Pain Remains
Processed: Grupo Frontera - En Vivo, Vol.1
Processed: Central Cee - Doja
Processed: Benson Boone - In The Stars
Processed: Luis Mexia - Haciendo Lo Mío
Processed: Kid Cudi - Entergalactic
Processed: Chase Atlantic - BEAUTY IN DEATH (DELUXE EDITION)
Processed: Jay Wheeler - Emociones
Processed: The Slumbering One - It's Raining
Processed: Juice WRLD - In My Head
Processed: Benny Friedman - It Sounds Like Chanukah
Processed: FKA twigs - CAPRISONGS
Processed: Electric Callboy - TEKKNO
Processed: Standly - Pégate
Processed: NAYEON - IM NAYEON
Processed: Rosa Linn - SNAP
Processed: Randy Houser - Note To Self
Proces

Processed: NAV - Demons Protected By Angels
Processed: Wonder Eve - Tis the season
Processed: Jeremih - Late Nights With Jeremih
Processed: Grupo Marca Registrada - El Rescate
Processed: 結束バンド - 結束バンド
Processed: Original Broadway Cast of & Juliet - & Juliet (Original Broadway Cast Recording)
Processed: Nate Smith - Whiskey On You
Processed: Coi Leray - Players
Processed: Patrick Watson - Ode to Vivian (Rework)
Processed: Gibran Alcocer - Idea 1
Processed: Blessd - Instagram
Processed: Paris Paloma - It's Called: Freefall
Processed: Creepy Nuts - アンサンブル・プレイ
Processed: KENTENSHI - paranoia
Processed: Eden Muñoz - Chale
Processed: Andrea Vanzo - Intimacy Vol. 1
Processed: Zander Jazz Trio - A Jazzy Christmas
Processed: SiM - The Rumbling
Processed: Hojean - Swing
Processed: Felix Jaehn - Call It Love
Processed: Maverick City Music - Kingdom Book One
Processed: Study Boy - Study Time
Processed: Grupo Origen - Cabron Yo Puedo
Processed: iñigo quintero - Si No Estás
Processed: iñigo quintero

Processed: Buddha's Lounge - Yoga Flute Meditation
Processed: David Guetta - Crazy What Love Can Do
Processed: MoonDeity - NEON BLADE
Processed: Jesus Image - JESUS (Live)
Processed: David Healer - Disney Piano Lullaby Best for Baby
Processed: Hollywood Undead - Hotel Kalifornia
Processed: Rels B - cómo dormiste?
Processed: Dalex - Yo Te Quiero Pa Mi
Processed: Oh so Tired - Everything's Wet
Processed: Snow Strippers - April Mixtape 2
Processed: Central Cee - LET GO
Processed: Tyga - Sunshine
Processed: Vince Guaraldi Trio - A Charlie Brown Christmas (2022 Mix)
Processed: Jax Jones - Where Did You Go (feat. MNEK)
Processed: Faceless 1-7 - Cut It Bleed It Breathe It
Processed: Tom Santa - Rainfall (Praise You)
Processed: Grupo Frontera - Si Yo Fuera Ladrón
Processed: 2hollis - White Tiger
Processed: Duskus - Let Go
Processed: David Schultz - Julmusik på piano (Vol. 2)
Processed: Alex Rose - ENR
Processed: Tom Odell - Best Day of My Life
Processed: Shakira - Te Felicito
Processed: Sofía 